In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! mkdir ~/.kaggle
#Copy the json kaggle to this directory
! cp kaggle.json ~/.kaggle/
#Allocate the required permission for this file.
! chmod 600 ~/.kaggle/kaggle.json
!pip install kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d plameneduardo/sarscov2-ctscan-dataset

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
 90% 208M/230M [00:01<00:00, 144MB/s]
100% 230M/230M [00:01<00:00, 126MB/s]


In [3]:
import os
import zipfile
#unzip_dataset
local_zip='/content/sarscov2-ctscan-dataset.zip'
zip_ref=zipfile.ZipFile(local_zip,'r')
zip_ref.extractall('/content/drive/MyDrive')

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential


In [5]:
data_dir = "/content/drive/MyDrive/sarscov2-ctscan-dataset"
img_height, img_width = 224, 224
batch_size = 64

In [6]:
data_dir =  '/content/drive/MyDrive/sarscov2-ctscan-dataset'
img_height, img_width = 224,224
batch_size = 64

train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  labels='inferred',
  label_mode='binary',
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  shuffle=True,
  interpolation='nearest',
  batch_size=batch_size)

valid_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  labels='inferred',
  label_mode='binary',
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  shuffle=True,
  interpolation='nearest',
  batch_size=batch_size)

Found 2481 files belonging to 2 classes.
Using 1985 files for training.
Found 2481 files belonging to 2 classes.
Using 496 files for validation.


In [7]:
rescale = tf.keras.Sequential([
    layers.Rescaling(1./255, input_shape=[img_height, img_width, 3])
])

In [8]:
model = keras.Sequential([

    rescale,

    # First Convolutional Block
    layers.Conv2D(filters=32, kernel_size=5, activation="relu", padding='same'),
    layers.MaxPool2D(),

    # Second Convolutional Block
    layers.Conv2D(filters=64, kernel_size=3, activation="relu", padding='same'),
    layers.MaxPool2D(),

    # Third Convolutional Block
    layers.Conv2D(filters=128, kernel_size=3, activation="relu", padding='same'),
    layers.MaxPool2D(),

    # Classifier Head
    layers.Flatten(),
    layers.Dense(units=128, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(units=64, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(units=32, activation="relu"),
    layers.Dense(units=1, activation="sigmoid"),
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 32)      2432      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 56, 56, 128)      

In [12]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds_preprocess = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
valid_ds_preprocess = valid_ds.cache().prefetch(buffer_size=AUTOTUNE)

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_ds_preprocess,
    validation_data=valid_ds_preprocess,
    epochs=50,
    verbose=1,
    callbacks=[callback],
)

Epoch 1/50
32/32 [==============================] - 18s 364ms/step - loss: 0.0782 - accuracy: 0.9698 - val_loss: 0.1709 - val_accuracy: 0.9395
Epoch 2/50
32/32 [==============================] - 3s 97ms/step - loss: 0.0568 - accuracy: 0.9819 - val_loss: 0.3463 - val_accuracy: 0.8952
Epoch 3/50
32/32 [==============================] - 3s 90ms/step - loss: 0.0701 - accuracy: 0.9713 - val_loss: 0.1754 - val_accuracy: 0.9315
Epoch 4/50
32/32 [==============================] - 3s 90ms/step - loss: 0.0734 - accuracy: 0.9783 - val_loss: 0.4162 - val_accuracy: 0.8851
Epoch 5/50
32/32 [==============================] - 3s 94ms/step - loss: 0.0530 - accuracy: 0.9773 - val_loss: 0.2151 - val_accuracy: 0.9194
Epoch 6/50
32/32 [==============================] - 3s 91ms/step - loss: 0.0416 - accuracy: 0.9844 - val_loss: 0.2237 - val_accuracy: 0.9355
Epoch 7/50
32/32 [==============================] - 3s 95ms/step - loss: 0.0478 - accuracy: 0.9834 - val_loss: 0.1717 - val_accuracy: 0.9456
Epoch 8/50


In [14]:

CLASSES = train_ds.class_names
probabilities = model.predict(valid_ds_preprocess)

predictions = [1 if i >0.5 else 0 for i in probabilities]
unbatched_valid_ds_preprocess = valid_ds_preprocess.unbatch()
labels_valid = [int(i[1]) for i in unbatched_valid_ds_preprocess]
list_labels_predictions = np.vstack((labels_valid,predictions)).T
conf_matrix = np.zeros([2,2])

for i,j in list_labels_predictions:
    conf_matrix[i,j] +=1

accuracy = accuracy_score(labels_valid, predictions)
f1_score = conf_matrix[1,1]/(conf_matrix[1,1] + ((conf_matrix[0,1] + conf_matrix[1,0])/2))
print('Val Accuracy: %.2f' % accuracy)
print('False positive: %.2f' % (conf_matrix[0,1]/len(predictions)))
print('False negative: %.2f' % (conf_matrix[1,0]/len(predictions)))
print('F1-score: %.2f' % (f1_score))



8/8 [==============================] - 0s 31ms/step
Val Accuracy: 0.93
False positive: 0.03
False negative: 0.04
F1-score: 0.93
